# 📁 020_silver/c4s_flow Documentation

This folder contains the pipeline responsible for preparing C4S (Customer For Sales) data, cleaning customer addresses, and enriching the records with **Google Maps Place IDs** using the Address Validation API. This geo-enrichment is critical for matching C4S accounts to Sellout customers later in the process.

## 🎯 **Core Function**

The pipeline performs incremental, rate-limited calls to the Google Address Validation API and maintains a persistent mapping table to ensure API limits are respected.

## 📄 **Notebook Summary and Dependencies**

| Notebook | Output Table | Function | Dependencies |
| :--- | :--- | :--- | :--- |
| **`01_c4s_dump`** | `01_c4s_dump` | **Data Preparation & Payload Generation:** Filters the raw Bronze C4S data to include only Belgian customers. It renames and selects core address columns and constructs the `addressvalidation` API payload column. | `d_dnst_010_bronze.csv_export.010_c4s_dump` |
| **`02_c4s_dump`** | `02_c4s_dump` | **API Filtering:** Reads the intermediate C4S data and checks against the persistent `google_api_mapping_table`. It calculates the remaining API calls permitted by the monthly limit and filters the C4S data to include only records needing a new Place ID lookup, limited by the remaining quota. | `d_dnst_020_silver.silver_01.01_c4s_dump`, `d_dnst_020_silver.silver_03.google_api_mapping_table` |
| **`03_c4s_api_call`** | `google_api_mapping_table` | **API Execution & Persistence (Merge):** Takes the filtered list from `02_c4s_dump`, executes the Google Address Validation API calls, and merges the results (including the `place_id` and raw JSON) into the persistent mapping table. This table uses the API payload as the primary merge key for idempotency. | `d_dnst_020_silver.silver_02.02_c4s_dump` |
| **`04_C4S_with_place_id`** | `02_C4S_with_place_id` | **Final Join:** Joins the original prepared C4S data (from `01_c4s_dump`) back to the updated `google_api_mapping_table` to retrieve the final, validated `place_id` for every customer record, completing the C4S flow preparation. | `d_dnst_020_silver.silver_01.01_c4s_dump`, `d_dnst_020_silver.silver_03.google_api_mapping_table` |

## ⚙️ **Key Features**

### 1. Persistent API Mapping

The `google_api_mapping_table` (created and maintained by `03_c4s_api_call`) acts as a cache. If the same customer address or payload is encountered again, the existing `place_id` is retrieved without a redundant API call.

### 2. Rate Limiting

The `02_c4s_dump` notebook dynamically calculates the number of API calls already made this month and uses `.limit(api_calls_left)` to throttle execution, ensuring the process stays within predefined external quota limits.

### 3. Delta Merge for Idempotency

`03_c4s_api_call` uses a `MERGE` statement to update existing records (in case the API response changed) or insert new ones into the mapping table, minimizing unnecessary rewrites and ensuring data integrity.